# PID Control

Perhaps the simplest way to control a multirotor is to use successive loop closure. This scheme creates inner and outer PID loops, as shown in the figure. The inner loop controls the attitude and the outer loop controls the position.

Using this scheme assumes gentle flight with small pitch and roll angles.

In [1]:
%%capture
# Load the quadrotor simulator from a previous notebook
%run quadsim.ipynb

In [2]:
class SimplePID:
    def __init__(self, kp, kd, ki, min, max, tau=0.05):
        self.kp = kp
        self.kd = kd
        self.ki = ki
        self.min = min
        self.max = max
        self.tau = tau

        self.derivative = 0.0
        self.integral = 0.0

        self.last_error = 0.0

    def run(self, error, dt, derivative=None):

        # P term
        if self.kp:
            p_term = self.kp * error
        else:
            p_term = 0.0

        # D term
        if self.kd:
            if not derivative is None:
                self.derivative = derivative
            elif dt > 0.0001:
                self.derivative = (2.0*self.tau - dt)/(2.0*self.tau + dt)*self.derivative + 2.0/(2.0*self.tau + dt)*(error - self.last_error)
            else:
                self.derivative = 0.0
            d_term = -self.kd * self.derivative
        else:
            d_term = 0.0

        # I term
        if self.ki:
            self.integral += error * dt
            i_term = self.ki * self.integral
        else:
            i_term = 0.0

        # combine
        u = p_term + d_term + i_term

        # saturate
        if u < self.min:
            u_sat = self.min
        elif u > self.max:
            u_sat = self.max
        else:
            u_sat = u

        # integrator anti-windup
        if self.ki:
            if abs(p_term + d_term) > abs(u_sat):
                # PD is already saturating, so set integrator to 0 but don't let it run backwards
                self.integral = 0
            else:
                # otherwise only let integral term at most take us just up to saturation
                self.integral = (u_sat - p_term - d_term) / self.ki

        # bookkeeping
        self.last_error = error

        return u_sat

In [5]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

class PID(Controller):
    """PID Controller
    """
    def __init__(self):
        self.name = "PID"
        
        # estimates of the physical properties of the quadrotor
        self.g = 9.81
        self.mass = 3.81
        Jxx = 0.060224; Jyy = 0.122198; Jzz = 0.132166
        # --- TEMPORARY ---
        self.g = 0
        self.mass = 1
        Jxx = Jyy = Jzz = 1
        # -----------------
        self.I = np.array([[Jxx,0,0],
                           [0,Jyy,0],
                           [0,0,Jzz]])
        
        # Attitude PID controllers
        self.pid_roll = SimplePID(2, None, 1, -2, 2)
        self.pid_pitch = SimplePID(1, None, None, -np.pi/6, np.pi/6)
        self.pid_yaw_rate = SimplePID(1, None, None, -np.pi, np.pi)
        
        # Position PID controllers
        self.pid_x = SimplePID(10, None, 20, -np.pi/12, np.pi/12)
        self.pid_y = SimplePID(1, None, None, -np.pi/12, np.pi/12)
        self.pid_z = SimplePID(2.0, None, None, -100, 100)
        self.pid_yaw = SimplePID(1, None, None, -np.pi/8, np.pi/8)
        
        # Compute feed-forward term for thrust
        self.z_ff = self.mass * self.g
        
        self.N = 0
               
    def inner(self, commanded, state, Ts):
        
        # flatten the inputs for easy access
        c = commanded.flatten()
        s = state.flatten()
        
        # Force (f) and moments (tau_*)
        f = c[0]
        tph = self.pid_roll.run(c[1]-s[6], Ts)
        tth = self.pid_pitch.run(c[2]-s[7], Ts)
        tps = self.pid_yaw_rate.run(c[3]-s[11], Ts)
        
        return [f, tph, tth, tps]
    
    def outer(self, commanded, state, Ts):
        
        # flatten the inputs for easy access
        c = commanded.flatten()
        s = state.flatten()
        
        th_c = -self.pid_x.run(c[0]-s[0], Ts)
        ph_c = self.pid_y.run(c[1]-s[1], Ts)
        thrust_c = self.pid_z.run(c[2]-s[2], Ts) + self.z_ff
        psdot_c = self.pid_yaw.run(c[3]-s[8], Ts)
        
        return np.array([thrust_c, ph_c, th_c, psdot_c])
    
    def update(self, commanded, state, Ts):
                
        #
        # Outer Loop
        #
        cmd = commanded[[0,1,2,8]]
        cmd = self.outer(cmd, state, Ts)
        
        #
        # Inner Loop
        #
        
#         cmd = np.array([cmd[0], 0, 0, 0])
        
        u = self.inner(cmd, state, Ts)
        
        # update the commanded states
        commanded[6] = cmd[1]
        commanded[7] = cmd[2]
        
        
        
#         from IPython.core.debugger import set_trace; set_trace()
#         commanded[2] = 5
#         thrust = -1*(commanded[2] - state[2]) + self.z_ff
#         u = np.array([thrust, 0, 0, 0])

        err = commanded[2] - state.flatten()[2]
        thrust_c = -1*(self.pid_z.run(err, Ts) - self.z_ff) # rotate 180 degrees
        u = np.array([thrust_c, 0.0, 0.0, 0.0])

#         print("({:.2f}): u: {:.4f}, error: {:.4f}".format(self.N*Ts, thrust_c, err))
        
        self.N += 1
    
        # actuator commands
        return u, commanded

In [ ]:
# Instantiate a quadrotor model with the given initial conditions
quad = Quadrotor(r=np.array([[0],[0],[5]]),
                 v=np.array([[0],[0],[0]]),
               Phi=np.array([[0],[0],[0]]))

# Instantiate a simple PID controller
ctrl = PID()

# Setup a setpoint commander
cmdr = Commander(default=True)

# Run the simulation
sim = Simulator(quad, ctrl, cmdr=cmdr)
sim.run(150, Ts=0.01)
sim.plot()

print(sim.quad)

(0.00): u: 10.0000, error: -5.0000
(0.01): u: 10.0000, error: -5.0000
(0.02): u: 9.9980, error: -4.9990
(0.03): u: 9.9940, error: -4.9970
(0.04): u: 9.9880, error: -4.9940
(0.05): u: 9.9800, error: -4.9900
(0.06): u: 9.9700, error: -4.9850
(0.07): u: 9.9580, error: -4.9790
(0.08): u: 9.9440, error: -4.9720
(0.09): u: 9.9281, error: -4.9640
(0.10): u: 9.9101, error: -4.9550
(0.11): u: 9.8901, error: -4.9451
(0.12): u: 9.8682, error: -4.9341
(0.13): u: 9.8443, error: -4.9221
(0.14): u: 9.8184, error: -4.9092
(0.15): u: 9.7905, error: -4.8953
(0.16): u: 9.7607, error: -4.8804
(0.17): u: 9.7290, error: -4.8645
(0.18): u: 9.6952, error: -4.8476
(0.19): u: 9.6595, error: -4.8298
(0.20): u: 9.6219, error: -4.8110
(0.21): u: 9.5824, error: -4.7912
(0.22): u: 9.5409, error: -4.7705
(0.23): u: 9.4975, error: -4.7488
(0.24): u: 9.4522, error: -4.7261
(0.25): u: 9.4050, error: -4.7025
(0.26): u: 9.3560, error: -4.6780
(0.27): u: 9.3050, error: -4.6525
(0.28): u: 9.2522, error: -4.6261
(0.29): u: 9